In [ ]:
%run -i ../examples/prepare_gp_optimizer.py

dofs = [kbv.x_rot, kbv.offz, kbh.x_rot, kbh.offz][:2]

hard_bounds = np.array([[-0.10, +0.10], [-0.50, +0.50], [-0.10, +0.10], [-0.50, +0.50]])[:2]

for dof in dofs:
    dof.kind = "hinted"

In [ ]:
gpo = bloptools.gp.load('model.gpo',
                        run_engine=RE, 
                        db=db, 
                        detector=w9, 
                        detector_type='image',
                        dofs=dofs, 
                        dof_bounds=hard_bounds, 
                        fitness_model='max_sep_density',)

In [ ]:
gpo.recommend(strategy='explore', greedy=False, n=8)

In [ ]:
plt.scatter(*gpo.recommend(strategy='exploit', greedy=False, n=8).T)

In [ ]:
gpo.plot_state(gridded=True)

In [ ]:
plt.scatter(*gpo.test_params.T, c=-bloptools.gp._negative_expected_information_gain(gpo.evaluator, gpo.validator, gpo.evaluator.X, gpo.params_trans_fun(gpo.test_params)[:,None]))

In [ ]:
plt.scatter(*gpo.params_trans_fun(gpo.test_params).T)
plt.scatter(*gpo.evaluator.X.T)

In [ ]:
%debug


In [ ]:
de = gpo.evaluator.copy()
dv = gpo.validator.copy()

In [ ]:
dv.c

In [ ]:
import torch

X2 = torch.cat([gpo.evaluator.torch_inputs, -gpo.evaluator.torch_inputs[-1][None]])
y2 = torch.cat([gpo.evaluator.torch_targets, -gpo.evaluator.torch_targets[-1][None]])

In [ ]:
X2

In [ ]:
de.model.set_train_data(X2, y2, strict=False)

In [ ]:
gpo.evaluator.model.train_targets.detach().numpy().astype(float)

In [ ]:
de.model.set_train_data

In [ ]:
gpo.recommend(strategy='explore', greedy=False, n=2)

In [ ]:
gpo.evaluator.copy()

In [ ]:
# gpo = Optimizer(
#     init_scheme='quasi-random', 
#     n_init=16, 
#     run_engine=RE, 
#     db=db, 
#     detector=w9, 
#     detector_type='image',
#     dofs=dofs, 
#     dof_bounds=hard_bounds, 
#     fitness_model='max_sep_density',
#     training_iter=100, 
#     verbose=True,
# )
# gpo.save('model.gpo')

In [ ]:
gpo.dummy_evaluator

In [ ]:
gpo.dummy_evaluator = bloptools.gp.GPR()
gpo.dummy_evaluator.set_data(gpo.evaluator.x, gpo.evaluator.y)
gpo.dummy_evaluator.model.load_state_dict(gpo.evaluator.model.state_dict())

In [ ]:
gpo.dummy_evaluator.regress

In [ ]:
explore_params = gpo.recommend(strategy='explore', greedy=False, n=8)

In [ ]:
plt.scatter(*gpo.test_params.T, c=-bloptools.gp._posterior_entropy(experimental_X=gpo.test_params[:,None]))
plt.scatter(*explore_params.T, c='r')

In [ ]:
gpo.test_params[:,None].shape

In [ ]:
%debug

In [ ]:
n_per_iter = 2

iter_params = np.random.uniform(size=(7, n_per_iter, gpo.n_dof)) * gpo.dof_bounds.ptp(axis=1) + gpo.dof_bounds.min(axis=1)

In [ ]:
current_entropy = gpo._posterior_entropy(None)
current_entropy

In [ ]:
def _reward(self, PARAMS, model='entropy'):
    
    if not PARAMS.ndim == 3: 
        raise ValueError
        
    return self._posterior_entropy() - self._posterior_entropy(PARAMS)


def _cost(self, PARAMS, model='normalized_distance'):
    
    if not PARAMS.ndim == 3: 
        raise ValueError
        
    costs = []
        
    for params in PARAMS:
        costs.append(np.sqrt(np.square(np.diff(np.r_[self.current_params[None], params.reshape(-1, self.n_dof)]  / gpo.dof_bounds.ptp(axis=1), axis=0)).sum()))

    return 1e-2 + np.array(costs)


def _loss(p, gpo, reward_model='entropy', cost_model='normalized_distance'):
    
    params = p.reshape(-1, gpo.n_dof)
    loss = -_reward(gpo, params[None]) / _cost(gpo, params[None])
    #print(loss)
    return loss


In [ ]:
_reward(gpo, iter_params), _cost(gpo, iter_params),  _objective(iter_params.ravel(), gpo)

In [ ]:
n_guide = 2

n_per_iter = 16

guide_points = np.random.uniform(size=(n_guide, gpo.n_dof)) * gpo.dof_bounds.ptp(axis=1) + gpo.dof_bounds.min(axis=1)

init_params = sp.interpolate.interp1d(np.arange(n_guide+1), 
                                      np.r_[gpo.current_params[None], guide_points], axis=0)(np.linspace(0, n_guide, n_per_iter))

bounds = np.repeat(gpo.dof_bounds[None], n_per_iter, axis=0).reshape(-1, 2)

In [ ]:
plt.scatter(*init_params.T)

In [ ]:
import scipy as sp
res = sp.optimize.minimize(_loss, 
                           x0=init_params.ravel(), 
                           args=(gpo,), 
                           bounds=bounds,
                           options={'maxiter': 16})
res.x.reshape(-1, gpo.n_dof)

In [ ]:
import torch
import warnings

In [ ]:
self = gpo


def _posterior_entropy(self, prior_params=None, experiment_params=None):
    
    """
    Returns the change in integrated entropy of the process contingent on some experiment using a Quasi-Monte Carlo integration over a
    dummy Gaussian processes.
    
    prior_params: numpy array with shape (n_params, n_dof)
    
    experiment_params: numpy array with shape (n_sets, n_params_per_set, n_dof)

    If None is passed, we return the posterior entropy of the real process.
    """
    
    if prior_params is None:
        prior_params = self.params  # one set of zero observations

    if experiment_params is None:
        experiment_params = np.empty((1, 0, self.n_dof))  # one set of zero observations

    if not experiment_params.ndim == 3:
        raise ValueError('Passed params must have shape (n_sets, n_params_per_set, n_dof).')

    # get the noise from the evaluator likelihood
    raw_noise = self.evaluator.model.state_dict()["likelihood.noise_covar.raw_noise"]
    noise = self.evaluator.model.likelihood.noise_covar.raw_noise_constraint.transform(raw_noise).item()

    # n_data is the number of points in each observation we consider (n_data = n_process + n_params_per_set)
    # x_data is an array of shape (n_sets, n_data, n_dof) that describes potential obervation states
    # x_star is an array of points at which to evaluate the entropy rate, to sum together for the QMCI
    x_data = torch.as_tensor(
        self.params_trans_fun(np.r_[[np.r_[prior_params, _p] for _p in np.atleast_3d(experiment_params)]])
    )
    x_star = torch.as_tensor(self.params_trans_fun(self.test_params))

    # for each potential observation state, compute the prior-prior and prior-posterior covariance matrices
    # $C_data_data$ is the covariance of the potential data with itself, for each set of obervations
    # $C_star_data$ is the covariance of the QMCI points with the potential data (n_sets, n_qmci, n_data)
    # we don't care about K_star_star for our purposes, only its diagonal which is a constant prior_variance
    C_data_data = self.evaluator.model.covar_module(x_data, x_data).detach().numpy().astype(
        float
    ) + noise**2 * np.eye(x_data.shape[1])
    C_star_data = self.evaluator.model.covar_module(x_star, x_data).detach().numpy().astype(float)

    prior_variance = self.evaluator.model.covar_module.output_scale.item() ** 2 + noise**2

    # normally we would compute A * B" * A', but that would be inefficient as we only care about the diagonal.
    # instead, compute this as:
    #
    # diag(A * B" * A') = sum(A * B" . A', -1)
    #
    # which is much faster.

    explained_variance = (np.matmul(C_star_data, np.linalg.inv(C_data_data)) * C_star_data).sum(axis=-1)
    posterior_variance = prior_variance - explained_variance

    n_bad, n_tot = (posterior_variance <= 0).sum(), len(posterior_variance.ravel())

    if not n_bad == 0:  # the posterior variance should always be positive
        warnings.warn(f"{n_bad}/{n_tot} variance estimates are non-positive.")
        if n_bad / n_tot > 0.5:
            raise ValueError("More than half of the variance estimates are non-positive.")

    marginal_entropy_rate = 0.5 * np.log(2 * np.pi * np.e * posterior_variance)

    return marginal_entropy_rate.sum(axis=-1)

In [ ]:
self = gpo

n_per_iter = 6

params_to_sample = np.zeros((0, self.n_dof))

for i in range(n_per_iter):
    
    print(i)

    PH = _posterior_entropy(self, prior_params=np.r_[self.params, params_to_sample], experiment_params=self.test_params[:,None])
    EdPH = self.validate(self.test_params) * (PH - _posterior_entropy(self))
    
    params_to_sample = np.r_[params_to_sample, self.test_params[np.nanargmin(EdPH)][None]]

In [ ]:
plt.scatter(*gpo.test_params.T, c=EdPH, cmap='gray')
plt.scatter(*params_to_sample.T, c='r')
plt.colorbar()

In [ ]:
params_to_sample

In [ ]:
gpo.test_params[EdPH.argmin()]

In [ ]:
plt.scatter(*gpo.params.T)
plt.scatter(*params_to_sample.T)

In [ ]:
plt.scatter(

In [ ]:
plt.scatter(*gpo.params.T)
plt.scatter(*init_params.T)
plt.plot(*res.x.reshape(-1, gpo.n_dof).T)

In [ ]:
res

In [ ]:
_cost(gpo, res.x.reshape(-1, gpo.n_dof)[None])

In [ ]:
current_params = gpo.current_params
current_params 

In [ ]:
dH = current_entropy - gpo._posterior_entropy(iter_params[None])

cost = 

In [ ]:
#gpo.evaluator.mll

In [ ]:
for i in range(4):

    gpo.learn(n_iter=1, n_per_iter=1, strategy='explore', greedy=True, reuse_hypers=False)
    gpo.learn(n_iter=1torch.square(self.output_scale[0]) * , n_per_iter=1, strategy='exploit', greedy=True, reuse_hypers=False)
#gpo.learn(n_iter=1, n_per_iter=1, strategy='D-optimal', greedy=True, reuse_hypers=False)

In [ ]:
FM = gpo._contingent_fisher_information_matrix(gpo.test_params[0])[0]
plt.imshow(np.abs(FM), norm=mpl.colors.LogNorm())
plt.colorbar()

In [ ]:
gpo.plot_fitness()